# Campbell Motors




## Setup Your Environment/Imports

In [ ]:
# before you do anything...
# mount your drive!
# click folder on the left...
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
        #!apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

assert(shutil.which("ipopt") or os.path.isfile("ipopt"))

from pyomo.environ import *

# Campbell Motors


Campbell Motors is an auto dealership that specializes in the sale of station wagons and light trucks.  Because of its reputation for quality and service, Campbell has a strong position in the regional market, but demand remains somewhat sensitive to price.  While evaluating the new models, Campbell's marketing consultant has come up with the following demand curves:
\begin{align*}
	TruckDemand & = 500 - 17 \cdot \frac{TruckPrice}{1,000} \\
	WagonDemand & = 400 - 12 \cdot \frac{WagonPrice}{1,000}
\end{align*}

The dealership's unit costs are $\$20,000$ for trucks and $\$25,000$ for wagons.  Each truck requires three hours of prep labor, and each wagon requires two hours of prep labor.  The current staff can supply 250 hours of labor.  Determine prices at which Campbell Motors can maximize the profit it generates from combined sales of trucks and wagons. For this exercise, both truck demand and wagon demand may assume non-integer values (i.e., you should not round demands in case you get fractional values).


**Optimal solution is $Truck Price = \$26,049$ and $Wagon Price = \$30,062$; optimal value (in thousands) is \$544,505.**




**Objective Function**

$\max t_d \cdot (t_p - 20,000 ) + w_d \cdot (w_p  - 25,000 )$ `(objective function)`

**Write the Constraints**

subject to:
* $t_d = 500 - 17 \cdot \frac{t_p}{1,000}$ `(truck demand)`
* $w_d = 400 - 12 \cdot \frac{w_p}{1,000} $ `(wagon demand)`
* $3t_d + 2w_d \leq 250$ `(prep labor)`

`Domains`
* $t_d,w_d \in \mathbb{R}^+$ `(demand variables)`
* $t_p,w_p \in \mathbb{R}^+$ `(price variables)`


In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables  - Initialize variable to 1.0
model.truck_price = Var(domain=NonNegativeReals,initialize=20000.0,bounds=(20000,1000*(500/17)))
model.wagon_price = Var(domain=NonNegativeReals,initialize=25000.0,bounds=(25000,1000*(400/12)))

model.truck_demand = Var(domain=NonNegativeReals,initialize=1.0,bounds=(0,500))
model.wagon_demand = Var(domain=NonNegativeReals,initialize=1.0,bounds=(0,400))


# Constraints
model.constraints = ConstraintList()

model.constraints.add(model.truck_demand == 500 - (17*model.truck_price)/1000)
model.constraints.add(model.wagon_demand == 400 - (12*model.wagon_price)/1000)
model.constraints.add(3*model.truck_demand  + 2*model.wagon_demand <= 250)

# declare objective
model.revenue = Objective(
                      expr = model.truck_demand*(model.truck_price - 20000) + model.wagon_demand*(model.wagon_price - 25000),
                      sense = maximize)

# show the model you've created
model.pprint()

1 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}

4 Var Declarations
    truck_demand : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :   500 : False : False : NonNegativeReals
    truck_price : Size=1, Index=None
        Key  : Lower : Value   : Upper              : Fixed : Stale : Domain
        None : 20000 : 20000.0 : 29411.764705882353 : False : False : NonNegativeReals
    wagon_demand : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :   400 : False : False : NonNegativeReals
    wagon_price : Size=1, Index=None
        Key  : Lower : Value   : Upper              : Fixed : Stale : Domain
        None : 25000 : 25000.0 : 33333.333333333336 : False : False : NonNegativeReals

1 Objective Declarations
    revenue : Size=1, Index=Non

In [ ]:
# solve it
SolverFactory('ipopt', executable='/content/ipopt').solve(model).write()
# show the results
print("Objective value = ", model.revenue())
print("Truck price = ", model.truck_price())
print("Truck demand = ", model.truck_demand())
print("Wagon price = ", model.wagon_price())
print("Wagon demand = ", model.wagon_demand())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 4
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.0374600887298584
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
Objectiv